**//IMPORTS**

In [4]:
import duckdb
import pandas as pd, pathlib, itertools, textwrap

**//CONFIGS**

In [5]:
DB_PATH = '../data/duckdb/subs.duckdb'
pd.set_option("display.max_colwidth", None)


**//MAIN CODE**

In [6]:
con = duckdb.connect(pathlib.Path('../data/duckdb/subs.duckdb'), read_only=True)
con.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='main'").fetchall()
con.close()


In [7]:
con = duckdb.connect(pathlib.Path('../data/duckdb/subs.duckdb'), read_only=False)
con.execute("SELECT * FROM opus_moses_filtered WHERE line_no = 79804").df()


,line_no,pair_id,sent_pt_br,sent_pt_pt
0,79804,79804,Temos uma deliciosa galinha d'Angola gelatinada com patê de fígado e trufas.,"Temos um peito muito bom de galinha da Guiné, senhor, em alfazema, com paté de foie gras e trufas."


**//LOAD TEST DATASETS**

In [8]:
# --- SETUP / CONNECT ---
import duckdb, pathlib, pandas as pd
pd.set_option("display.max_colwidth", 180)

PROJECT_DB_PATH = pathlib.Path("../data/duckdb/subs_project.duckdb")
SOURCE_DB_PATH  = pathlib.Path("../data/duckdb/subs.duckdb")

PROJECT_DB_PATH.parent.mkdir(parents=True, exist_ok=True)
con = duckdb.connect(PROJECT_DB_PATH.as_posix())   # read/write OK

# Attach source DB (idempotent)
try:
    con.execute(f"ATTACH '{SOURCE_DB_PATH.as_posix()}' AS src")
except duckdb.BinderException:
    pass

def tbl_exists(q: str) -> bool:
    try:
        con.execute(f"SELECT 1 FROM {q} LIMIT 1")
        return True
    except Exception:
        return False

def lo(c: str) -> str:
    return f"lower(coalesce({c},''))"

print("✓ Connected. src attached:", SOURCE_DB_PATH.exists())

# Find the ptbrvarid source table in subs.duckdb
src_ptbr = None
for cand in ["src.main.ptbrvarid", "src.ptbrvarid", "src.main.PtBrVarId", "src.PtBrVarId"]:
    if tbl_exists(cand):
        src_ptbr = cand
        break
if not src_ptbr:
    raise RuntimeError("Cannot find ptbrvarid in subs.duckdb (tried src.main.ptbrvarid, src.ptbrvarid, ...).")

print("Using ptbrvarid source:", src_ptbr)


✓ Connected. src attached: True
Using ptbrvarid source: src.main.ptbrvarid


In [9]:
from datasets import load_dataset

# Load HF splits (each has 500 rows with a "text" field)
bp     = load_dataset("joaosanches/golden_collection", split="gold_collection")
manual = load_dataset("joaosanches/golden_collection", split="referencia_manual")
deepl  = load_dataset("joaosanches/golden_collection", split="referencia_DeepL")
assert len(bp)==len(manual)==len(deepl)==500, (len(bp),len(manual),len(deepl))

# Create target and DELETE everything first
con.execute("""
CREATE TABLE IF NOT EXISTS gold_test (
  bucket TEXT,
  theme  TEXT,
  text_pt_br TEXT,
  ref_pt_pt_manual TEXT,
  ref_pt_pt_deepl  TEXT
);
""")
con.execute("DELETE FROM gold_test;")

def _clean(s: str) -> str:
    if s is None: return None
    return " ".join(str(s).replace("\r"," ").replace("\n"," ").split()).strip()

rows = [
    ("n/a", "n/a", _clean(bp[i]["text"]), _clean(manual[i]["text"]), _clean(deepl[i]["text"]))
    for i in range(500)
]
con.executemany(
    "INSERT INTO gold_test (bucket, theme, text_pt_br, ref_pt_pt_manual, ref_pt_pt_deepl) VALUES (?,?,?,?,?)",
    rows
)

# Sanity on gold
display(con.execute("""
SELECT
  (SELECT COUNT(*) FROM gold_test) AS gold_rows,
  (SELECT COUNT(DISTINCT lower(text_pt_br)) FROM gold_test) AS distinct_bp,
  (SELECT SUM(CASE WHEN coalesce(ref_pt_pt_manual,'')='' THEN 1 ELSE 0 END) FROM gold_test) AS null_manual,
  (SELECT SUM(CASE WHEN coalesce(ref_pt_pt_deepl ,'')='' THEN 1 ELSE 0 END) FROM gold_test) AS null_deepl
""").df())


,gold_rows,distinct_bp,null_manual,null_deepl
0,500,500,0.0,0.0


In [10]:
con.execute("DROP VIEW IF EXISTS ptbrvid_repaired_v;")
con.execute("""
CREATE VIEW ptbrvid_repaired_v AS
WITH raw AS (
  SELECT dataset, domain, split, label, text_pt_br, text_pt_pt
  FROM src.main.ptbrvarid
  WHERE dataset='PtBrVId'
),
norm AS (
  SELECT
    -- language
    CASE
      WHEN lower(label) IN ('pt-br','pt-pt') THEN CASE WHEN lower(label)='pt-br' THEN 'pt-BR' ELSE 'pt-PT' END
      WHEN text_pt_br IN ('pt-BR','pt-PT')      THEN text_pt_br
      ELSE NULL
    END AS lang,

    -- text (prefer proper columns; fall back to domain only if it looks like text)
    CASE
      WHEN text_pt_br IS NOT NULL AND text_pt_br NOT IN ('pt-BR','pt-PT') THEN text_pt_br
      WHEN text_pt_pt IS NOT NULL AND text_pt_pt NOT IN ('pt-BR','pt-PT') THEN text_pt_pt
      WHEN domain IS NOT NULL
           AND lower(domain) NOT IN ('journalistic','legal','web','literature','politics','social_media')
           AND length(domain) > 40 THEN domain
      ELSE NULL
    END AS text,

    split, domain
  FROM raw
)
SELECT
  'PtBrVId' AS dataset,
  split,
  lang  AS label,
  CASE WHEN lang='pt-BR' THEN text END AS text_pt_br,
  CASE WHEN lang='pt-PT' THEN text END AS text_pt_pt
FROM norm
WHERE lang IS NOT NULL AND text IS NOT NULL;
""")

print(con.execute("SELECT COUNT(*) AS repaired_rows FROM ptbrvid_repaired_v").df())
print(con.execute("SELECT label, COUNT(*) AS n FROM ptbrvid_repaired_v GROUP BY 1 ORDER BY n DESC").df())


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

   repaired_rows
0        2991704
   label        n
0  pt-PT  2654553
1  pt-BR   337151


In [ ]:
con.execute("DROP VIEW IF EXISTS ptbr_unique_v;")
con.execute("""
CREATE VIEW ptbr_unique_v AS
WITH keyed AS (
  SELECT
    label,
    NULLIF(TRIM(text_pt_br),'') AS text_pt_br,
    NULLIF(TRIM(text_pt_pt),'') AS text_pt_pt,
    lower(coalesce(text_pt_br, text_pt_pt, '')) AS k_txt
  FROM ptbrvid_repaired_v
),
clean AS (
  SELECT label, text_pt_br, text_pt_pt, k_txt
  FROM keyed
  WHERE k_txt <> ''
),
agg AS (
  SELECT
    label, k_txt,
    arg_max(text_pt_br, length(coalesce(text_pt_br,''))) AS text_pt_br,
    arg_max(text_pt_pt, length(coalesce(text_pt_pt,''))) AS text_pt_pt
  FROM clean
  GROUP BY label, k_txt
)
SELECT label, text_pt_br, text_pt_pt
FROM agg;
""")

con.execute("DROP VIEW IF EXISTS ptbr_split_assign_v;")
con.execute("""
CREATE VIEW ptbr_split_assign_v AS
WITH base AS (
  SELECT
    label,
    text_pt_br,
    text_pt_pt,
    lower(coalesce(text_pt_br, text_pt_pt, '')) AS k_txt
  FROM ptbr_unique_v
)
SELECT
  CASE WHEN (hash(k_txt, coalesce(label,'')) % 5) = 0 THEN 'test' ELSE 'train' END AS split,
  label, text_pt_br, text_pt_pt
FROM base;
""")

print(con.execute("""
SELECT split, COUNT(*) AS n
FROM ptbr_split_assign_v
GROUP BY 1
ORDER BY 2 DESC
""").df())


In [ ]:
con.execute("DROP VIEW IF EXISTS ptbr_split_assign_v;")
con.execute("""
CREATE VIEW ptbr_split_assign_v AS
WITH base AS (
  SELECT
    label,
    text_pt_br,
    text_pt_pt,
    lower(coalesce(text_pt_br, text_pt_pt, '')) AS k_txt
  FROM ptbr_unique_v
)
SELECT
  CASE WHEN (hash(k_txt, coalesce(label,'')) % 5) = 0 THEN 'test' ELSE 'train' END AS split,
  label, text_pt_br, text_pt_pt
FROM base;
""")

print(con.execute("""
WITH c AS (
  SELECT SUM(CASE WHEN split='test' THEN 1 ELSE 0 END) AS n_test,
         COUNT(*) AS n_total
  FROM ptbr_split_assign_v
)
SELECT n_test, n_total, ROUND(100.0*n_test/NULLIF(n_total,0),2) AS pct_test FROM c;
""").df())


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

     n_test  n_total  pct_test
0  597931.0  2990689     19.99


In [ ]:
# Optional FRMT/GOLD test-pair guard
def have(q: str) -> bool:
    return tbl_exists(q)

con.execute("DROP VIEW IF EXISTS test_pairs_guard;")
have_frmt = have("frmt_test")
have_gold = have("gold_test")

if have_frmt and have_gold:
    con.execute(f"""
      CREATE VIEW test_pairs_guard AS
      SELECT DISTINCT n_br, n_pt FROM (
        SELECT {lo('text_pt_br')} AS n_br, {lo('text_pt_pt')} AS n_pt FROM frmt_test
        UNION ALL
        SELECT {lo('text_pt_br')} AS n_br, {lo('ref_pt_pt_manual')} AS n_pt FROM gold_test
      );
    """)
elif have_frmt:
    con.execute(f"""
      CREATE VIEW test_pairs_guard AS
      SELECT DISTINCT {lo('text_pt_br')} AS n_br, {lo('text_pt_pt')} AS n_pt FROM frmt_test;
    """)
elif have_gold:
    con.execute(f"""
      CREATE VIEW test_pairs_guard AS
      SELECT DISTINCT {lo('text_pt_br')} AS n_br, {lo('ref_pt_pt_manual')} AS n_pt FROM gold_test;
    """)
else:
    con.execute("CREATE VIEW test_pairs_guard AS SELECT ''::TEXT AS n_br, ''::TEXT AS n_pt WHERE 1=0;")

print("test_pairs_guard rows:", con.execute("SELECT COUNT(*) FROM test_pairs_guard").fetchone()[0])


test_pairs_guard rows: 3108


In [ ]:
def has_cols(q: str, want=("sent_pt_br","sent_pt_pt")) -> bool:
    try:
        cols = con.execute(f"DESCRIBE {q}").df()["column_name"].str.lower().tolist()
        return all(w in cols for w in want)
    except Exception:
        return False

candidates = [
    "src.main.opus_moses_filtered", "main.opus_moses_filtered",
    "src.main.opus_filtered",       "main.opus_filtered",
    "src.main.opus_filter_simple",  "main.opus_filter_simple",
    "src.main.opus_moses",          "main.opus_moses",
]
chosen = None
for q in candidates:
    if tbl_exists(q) and has_cols(q):
        chosen = q; break

con.execute("DROP VIEW IF EXISTS opus_source;")
if chosen:
    con.execute(f"CREATE VIEW opus_source AS SELECT * FROM {chosen};")
    print(f"✓ opus_source -> {chosen}")
else:
    con.execute("CREATE VIEW opus_source AS SELECT NULL::TEXT AS sent_pt_br, NULL::TEXT AS sent_pt_pt WHERE 1=0;")
    print("! No OPUS table found with (sent_pt_br, sent_pt_pt)")


✓ opus_source -> src.main.opus_moses_filtered


In [ ]:
# TRAIN view
con.execute("""
CREATE OR REPLACE VIEW train_data AS
-- OPUS (anti-join: don't leak known test pairs)
SELECT
  'OpenSubs' AS dataset, 'opus_source' AS source,
  'n/a' AS bucket, 'n/a' AS theme,
  CAST(NULL AS TEXT) AS label,
  o.sent_pt_br AS text_pt_br, o.sent_pt_pt AS text_pt_pt,
  CAST(NULL AS TEXT) AS ref_pt_pt_manual, CAST(NULL AS TEXT) AS ref_pt_pt_deepl
FROM opus_source o
LEFT JOIN test_pairs_guard g
  ON lower(coalesce(o.sent_pt_br,'')) = g.n_br
 AND lower(coalesce(o.sent_pt_pt,'')) = g.n_pt
WHERE g.n_br IS NULL

UNION ALL
-- PtBrVarId (assigned TRAIN, ~80%)
SELECT
  'PtBrVarId' AS dataset, 'liaad/PtBrVId' AS source,
  'n/a' AS bucket, 'n/a' AS theme,
  p.label,
  p.text_pt_br, p.text_pt_pt,
  CAST(NULL AS TEXT), CAST(NULL AS TEXT)
FROM ptbr_split_assign_v p
WHERE lower(p.split) = 'train'

UNION ALL
-- FRMT dev (if present)
SELECT
  'FRMT' AS dataset, 'google-research/frmt' AS source,
  f.bucket, 'n/a' AS theme, CAST(NULL AS TEXT) AS label,
  f.text_pt_br, f.text_pt_pt,
  CAST(NULL AS TEXT), CAST(NULL AS TEXT)
FROM frmt_dev f;
""")

# TEST view
con.execute("""
CREATE OR REPLACE VIEW test_data AS
-- PtBrVarId (assigned TEST, ~20%)
SELECT
  'PtBrVarId' AS dataset, 'liaad/PtBrVId' AS source,
  'n/a' AS bucket, 'n/a' AS theme, p.label,
  p.text_pt_br, p.text_pt_pt,
  CAST(NULL AS TEXT) AS ref_pt_pt_manual, CAST(NULL AS TEXT) AS ref_pt_pt_deepl
FROM ptbr_split_assign_v p
WHERE lower(p.split) = 'test'

UNION ALL
-- FRMT test (if present)
SELECT
  'FRMT' AS dataset, 'google-research/frmt' AS source,
  f.bucket, 'n/a' AS theme, CAST(NULL AS TEXT) AS label,
  f.text_pt_br, f.text_pt_pt,
  CAST(NULL AS TEXT), CAST(NULL AS TEXT)
FROM frmt_test f

UNION ALL
-- Gold (500 rows)
SELECT
  'Gold' AS dataset, 'joaosanches/golden_collection' AS source,
  COALESCE(g.bucket,'n/a'), COALESCE(g.theme,'n/a'),
  'n/a' AS label,
  g.text_pt_br, CAST(NULL AS TEXT) AS text_pt_pt,
  g.ref_pt_pt_manual, g.ref_pt_pt_deepl
FROM gold_test g;
""")


In [ ]:
import duckdb, pathlib

PROJECT_DB_PATH = pathlib.Path("../data/duckdb/subs_project.duckdb")

con = duckdb.connect(PROJECT_DB_PATH.as_posix())

# Just to see that the views exist
print(con.execute("""
SELECT table_name, table_type
FROM information_schema.tables
WHERE table_name IN ('train_data','test_data')
""").df())

# Drop old materializations if they exist
con.execute("DROP TABLE IF EXISTS train_data_instance;")
con.execute("DROP TABLE IF EXISTS test_data_instance;")

# Materialize the current views
con.execute("CREATE TABLE train_data_instance AS SELECT * FROM train_data;")
con.execute("CREATE TABLE test_data_instance  AS SELECT * FROM test_data;")

print("Rows in train_data_instance:",
      con.execute("SELECT COUNT(*) FROM train_data_instance").fetchone()[0])
print("Rows in test_data_instance :",
      con.execute("SELECT COUNT(*) FROM test_data_instance").fetchone()[0])

con.close()


   table_name  table_type
0   test_data  BASE TABLE
1  train_data        VIEW
2   test_data        VIEW
3  train_data        VIEW


In [ ]:
# What FRMT relations exist?
frmt_present = {
    "frmt_dev": tbl_exists("frmt_dev"),
    "frmt_test": tbl_exists("frmt_test")
}
display(pd.DataFrame([frmt_present]))

# Counts in FRMT base tables (if present)
if frmt_present["frmt_dev"]:
    print("frmt_dev rows:", con.execute("SELECT COUNT(*) FROM frmt_dev").fetchone()[0])
if frmt_present["frmt_test"]:
    print("frmt_test rows:", con.execute("SELECT COUNT(*) FROM frmt_test").fetchone()[0])

# Where FRMT surfaced in unified views:
print("\nFRMT in train_data:")
display(con.execute("""
SELECT COUNT(*) AS n
FROM train_data
WHERE dataset='FRMT'
""").df())

print("FRMT in test_data:")
display(con.execute("""
SELECT COUNT(*) AS n
FROM test_data
WHERE dataset='FRMT'
""").df())

# Optional: peek a few rows with an inferred origin label (dev/test) based on which view they’re in
print("\nSample FRMT rows from train_data (origin ~ FRMT-dev):")
display(con.execute("""
SELECT dataset, text_pt_br, text_pt_pt
FROM train_data
WHERE dataset='FRMT'
LIMIT 5
""").df())

print("\nSample FRMT rows from test_data (origin ~ FRMT-test):")
display(con.execute("""
SELECT dataset, text_pt_br, text_pt_pt
FROM test_data
WHERE dataset='FRMT'
LIMIT 5
""").df())


,frmt_dev,frmt_test
0,True,True


frmt_dev rows: 2521
frmt_test rows: 2608

FRMT in train_data:


,n
0,2521


FRMT in test_data:


,n
0,2608



Sample FRMT rows from train_data (origin ~ FRMT-dev):


,dataset,text_pt_br,text_pt_pt
0,FRMT,"Um telefone móvel, telefone celular, telefone portátil, muitas vezes chamado apenas de celular ou de telefone, é um telefone portátil que faz e recebe chamadas através de um ca...","Um telefone móvel, telefone celular, telefone manual, abreviado para telemóvel, é um telefone portátil que pode fazer e receber chamadas através de uma ligação de radiofrequênc..."
1,FRMT,"O link da frequência de rádio estabelece uma conexão com os sistemas variáveis de uma operadora de celular, o que dá acesso à rede pública de telefonia.",A radiofrequência estabelece uma ligação aos sistemas de comutação de uma operadora de telemóveis que permite o acesso à rede telefónica pública comutada (RTPC).
2,FRMT,"Os serviços modernos de telefonia celular usam uma arquitetura telefônica celular, e, assim, telefones portáteis são chamados de telefones celulares ou ""cell phones"" na América...","Os serviços de telemóveis modernos utilizam uma arquitetura de rede celular e, por conseguinte, em Portugal, chamam-se apenas telemóveis."
3,FRMT,"Além da telefonia, os aparelhos dos anos 2000 fornecem uma variedade de outros serviços, como mensagens de texto, MMS, e-mail, acesso à internet, comunicação sem fio de curto a...","Além da telefonia, os telemóveis da década de 2000 estão equipados com vários outros serviços, como mensagens de texto, MMS, e-mail, acesso à Internet, comunicações sem fios de..."
4,FRMT,"Telefones celulares que oferecem esses outros serviços são chamados de ""feature phones"", os que têm capacidades de computação ainda mais avançadas são chamados de smartphones.","Os telemóveis que apresentam apenas estas capacidades, são conhecidos como telemóveis básicos. Os que incluem capacidades de cálculo muito avançadas são conhecidos como smartph..."



Sample FRMT rows from test_data (origin ~ FRMT-test):


,dataset,text_pt_br,text_pt_pt
0,FRMT,"Um ônibus (contração de ômnibus, com as variações multibus, motorbus, autobus etc.) é um veículo rodoviário projetado para transportar muitos passageiros.",Um autocarro é um veículo rodoviário concebido para transportar muitos passageiros.
1,FRMT,Ônibus podem ter capacidade para até 300 passageiros.,Os autocarros têm capacidade para transportar no máximo 300 passageiros.
2,FRMT,"O tipo de ônibus mais comum é o ônibus rígido de um andar, com cargas maiores transportadas por ônibus de dois andares e articulados, e cargas menores transportadas por ônibus ...","Os tipos de autocarros mais comuns são os rígidos de um andar, sendo as cargas maiores nos de dois andares e os autocarros articulados. As cargas menores transportadas por auto..."
3,FRMT,"Muitos tipos de ônibus, como os de transporte urbano e os intermunicipais, cobram uma tarifa.","Muitos tipos de autocarros, como os de trânsito urbano e intercidades, cobram bilhete."
4,FRMT,"Outros tipos, como ônibus escolares do ensino médio ou fundamental ou ônibus de traslado em uma instituição de ensino superior não cobram tarifas.","Outros tipos, como os autocarros da escola primária ou secundária ou de ida e volta num campus pós-secundário não cobram bilhete."


**// Statistics**

In [ ]:
import duckdb, pathlib, pandas as pd
pd.set_option("display.max_colwidth", 180)

PROJECT_DB_PATH = pathlib.Path("../data/duckdb/subs_project.duckdb")
con = duckdb.connect(PROJECT_DB_PATH.as_posix())  # read/write OK

def tbl_exists(name: str) -> bool:
    try:
        con.execute(f"SELECT 1 FROM {name} LIMIT 1"); return True
    except Exception:
        return False

def describe_relation(name: str) -> pd.DataFrame:
    try:
        return con.execute(f"DESCRIBE {name}").df()
    except Exception as e:
        return pd.DataFrame({"error":[str(e)]})

def count_rows(name: str) -> int:
    try:
        return con.execute(f"SELECT COUNT(*) FROM {name}").fetchone()[0]
    except Exception:
        return 0

def lo(c: str) -> str:
    return f"lower(coalesce({c},''))"

def random_sample(name: str, n: int = 20) -> pd.DataFrame:
    try:
        return con.execute(f"SELECT * FROM {name} USING SAMPLE {n} ROWS").df()
    except Exception:
        return con.execute(f"SELECT * FROM {name} ORDER BY random() LIMIT {n}").df()


In [ ]:
candidates = [
    "train_data", "test_data",
    "ptbrvarid_split_assign", "ptbrvarid_v",
    "opus_source", "frmt_dev", "frmt_test",
    "gold_test"
]
present = [r for r in candidates if tbl_exists(r)]

summary_rows = []
for name in present:
    desc = describe_relation(name)
    cols = ", ".join(desc["column_name"].tolist()) if "column_name" in desc else "(n/a)"
    # find type from information_schema
    rtype = con.execute("""
        SELECT table_type FROM information_schema.tables
        WHERE table_schema='main' AND table_name=?
    """,[name]).fetchone()
    summary_rows.append({
        "relation": name,
        "type": (rtype[0] if rtype else "UNKNOWN"),
        "n_rows": count_rows(name),
        "columns": cols
    })

summary_df = pd.DataFrame(summary_rows).sort_values("relation")
summary_df


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,relation,type,n_rows,columns
4,frmt_dev,BASE TABLE,2521,"en, text_pt_br, text_pt_pt, bucket"
5,frmt_test,BASE TABLE,2608,"en, text_pt_br, text_pt_pt, bucket"
6,gold_test,BASE TABLE,500,"bucket, theme, text_pt_br, ref_pt_pt_manual, ref_pt_pt_deepl"
3,opus_source,VIEW,10347894,"line_no, pair_id, sent_pt_br, sent_pt_pt"
2,ptbrvarid_v,VIEW,337128,"split, label, text_pt_br, text_pt_pt"
1,test_data,BASE TABLE,601039,"dataset, source, bucket, theme, label, text_pt_br, text_pt_pt, ref_pt_pt_manual, ref_pt_pt_deepl"
0,train_data,VIEW,12743162,"dataset, source, bucket, theme, label, text_pt_br, text_pt_pt, ref_pt_pt_manual, ref_pt_pt_deepl"


In [ ]:
bd_train = con.execute("""
SELECT dataset, COUNT(*) AS n
FROM train_data
GROUP BY 1 ORDER BY n DESC
""").df() if tbl_exists("train_data") else pd.DataFrame()

bd_test = con.execute("""
SELECT dataset, COUNT(*) AS n
FROM test_data
GROUP BY 1 ORDER BY n DESC
""").df() if tbl_exists("test_data") else pd.DataFrame()

bd_train, bd_test


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

(     dataset         n
 0   OpenSubs  10347883
 1  PtBrVarId   2392758
 2       FRMT      2521,
      dataset       n
 0  PtBrVarId  597931
 1       FRMT    2608
 2       Gold     500)

In [ ]:
# Random 20 from train_data and test_data
train20 = con.execute("""
    SELECT * FROM train_data
    ORDER BY random()
    LIMIT 20
""").df()

test20 = con.execute("""
    SELECT * FROM test_data 
    ORDER BY random()
    LIMIT 20
""").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
test20

,dataset,source,bucket,theme,label,text_pt_br,text_pt_pt,ref_pt_pt_manual,ref_pt_pt_deepl
0,PtBrVarId,liaad/PtBrVId,n/a,n/a,pt-PT,None,"Como verdadeiro tribunal de substituição, a Relação aprecia livremente as provas produzidas segundo a sua prudente convicção acerca de cada facto impugnado, exceto no que respe...",None,None
1,PtBrVarId,liaad/PtBrVId,n/a,n/a,pt-PT,None,"Lembra-se, a propósito, que, no documento sobre a ""reestruturação geral da empresa"", em que a Administração se comprometia a ""reestruturar, redimensionar, reinstalar e reequipa...",None,None
2,PtBrVarId,liaad/PtBrVId,n/a,n/a,pt-PT,None,"Entendemos ser de reconhecer, relativamente às necessidades da menor, que a mesma entrará numa fase do seu desenvolvimento em que o aumento de despesas se faz sentir com mais i...",None,None
3,PtBrVarId,liaad/PtBrVId,n/a,n/a,pt-BR,"Entre o final dos anos 40 e começo dos 50, era capaz de tudo para chamar atenção em público. Ficou famosa por seus números nas revistas do Teatro Recreio, do empresário Walter ...",None,None,None
4,PtBrVarId,liaad/PtBrVId,n/a,n/a,pt-PT,None,"Além disso, entende-se que o conhecimento do mérito da causa, total ou parcialmente, só deve ter lugar quando o processo contenha todos os elementos necessários para uma decisã...",None,None
5,PtBrVarId,liaad/PtBrVId,n/a,n/a,pt-PT,None,"É que o CSC separa, respetivamente a remuneração da pensão de reforma, nos arts.39 e 40; -a atribuição da remuneração é fixada por deliberação da assembleia geral,enquanto que ...",None,None
6,PtBrVarId,liaad/PtBrVId,n/a,n/a,pt-PT,None,"Se votarem ""sim"", e todas as sondagens apontam para que isso aconteça, então Londres vai passar a ser a primeira cidade da Grã-Bretanha a ter um presidente da câmara. Alguém qu...",None,None
7,PtBrVarId,liaad/PtBrVId,n/a,n/a,pt-BR,"José Dirceu disse que a exploração do fato é uma tentativvva de desestabilizar a campanha de Lula. ""Eu e a Erundina também usamos o carro de som e ninguém falou nada"", disse. P...",None,None,None
8,PtBrVarId,liaad/PtBrVId,n/a,n/a,pt-PT,None,"Tão sensível foi o encontro do rei com a imprensa local que o jornal árabe ""Al-Hayat"", impresso em Londres, viu ontem confiscados os seus exemplares distribuídos em Amã, por te...",None,None
9,PtBrVarId,liaad/PtBrVId,n/a,n/a,pt-PT,None,"Ora, no caso dos autos, a ampliação pretendida pelo recorrente, no sentido da condenação da Autora no pagamento desse novo montante, quando formulou pedido de condenação em €24...",None,None
